# Landscape Singlet Lens

Let's design a simple landscape singlet lens. This means two things:

* A single lens (two refractive surfaces) made of one material: `tlm.AsymmetricLens`
* A light source at infinity: `tlm.ObjectAtInfinity`

We will also add an Aperture to model the lens entrance pupil. In the 'rear' case, the light source element acts as the aperture. In the 'front' case, an aperture is added behind the lens.

## Landscape rear configuration

In [ ]:
import torch
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
shape1 = tlm.CircularArc(height=30, r=tlm.Parameter(torch.tensor(-65.)))
shape2 = tlm.CircularArc(height=30, r=tlm.Parameter(torch.tensor(-25.)))

# Build the optical sequence
optics = tlm.OpticalSequence(
    tlm.ObjectAtInfinity(beam_diameter=10, angular_size=40),
    tlm.Gap(15),
    tlm.AsymmetricLens(shape1, shape2, (1.0, 1.5), outer_thickness=3.),
    tlm.Gap(120),
    tlm.ImagePlane(height=100),
)

# Render the optical configuration with initial values
# Sample 10 rays per location, and 5 object locations: total = 50 rays
tlm.render_plt(optics, color_dim="object", sampling={"rays": 10, "object": 5})
tlm.plot_magnification(optics, sampling={"rays": 10, "object": 5})

# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"rays": 10, "object": 10},
    num_iter = 100,
)

# Render with final values
tlm.render_plt(optics, sampling={"rays": 10, "object": 5}, color_dim="object")
tlm.plot_magnification(optics, sampling={"rays": 10, "object": 5})

## Landscape front configuration

In [ ]:
import torch
import torchlensmaker as tlm
import torch.optim as optim

# Setup two spherical surfaces with initial radiuses
shape1 = tlm.CircularArc(height=30, r=tlm.Parameter(torch.tensor(25.)))
shape2 = tlm.CircularArc(height=30, r=tlm.Parameter(torch.tensor(65.)))

lens = tlm.AsymmetricLens(shape1, shape2, (1.0, 1.5), outer_thickness=3.)

# Build the optical sequence
optics = tlm.OpticalSequence(
    tlm.ObjectAtInfinity(beam_diameter=40, angular_size=40),
    tlm.Gap(15),
    lens,
    tlm.Gap(20),
    tlm.Aperture(height=50, diameter=10),
    tlm.Gap(100),
    tlm.ImagePlane(height=100),
)

# Render the optical configuration with initial values
# Sample 10 rays per location, and 5 object locations: total = 50 rays
tlm.render_plt(optics, color_dim="object", sampling={"rays": 10, "object": 5})
tlm.plot_magnification(optics, sampling={"rays": 10, "object": 5})

# Find the best parameters for the shapes
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"rays": 10, "object": 10},
    num_iter = 200,
)

# Render with final values
tlm.render_plt(optics, sampling={"rays": 30, "object": 5}, color_dim="object")
tlm.plot_magnification(optics, sampling={"rays": 10, "object": 5})

In [ ]:
from IPython.display import display
import build123d as bd

part = tlm.lens_to_part(lens)
display(part)